In [3]:
%load_ext autoreload

In [4]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]=""

In [20]:
import pandas as pd
from tqdm import tqdm
import torch
import sys
import time
import datetime
from itertools import product
import os
import numpy as np
from sklearn import tree
from sklearn import linear_model
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, f1_score
import matplotlib.pyplot as plt
from functools import partial
import shelve
import tqdm
import pickle
from collections import Counter

In [5]:
from ogb.graphproppred import Evaluator
from tdc.benchmark_group import admet_group

/home/michlampert/magisterka/venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
%autoreload 2
from huggingmolecules import MatConfig, MatFeaturizer, MatModel
from huggingmolecules import GroverConfig, GroverFeaturizer, GroverModel
from huggingmolecules import RMatConfig, RMatFeaturizer, RMatModel
from models import StrippedMatModel, StrippedGroverModel, StrippedRMatModel

In [7]:
%autoreload 2
from utils import *
from datasets import *
from predict_procedure import *

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Error: Unable to import pysam. Please make sure it is installed.


Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'


Error: Unable to import pysam. Please make sure it is installed.


Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [8]:
# from mat import MolecularAttentionTransformer

# model = MolecularAttentionTransformer()
# model.predict(["CN1CC[C@]23C(=O)C[C@@H]4C(=CCO[C@H]5CC(=O)N(c6ccccc62)[C@@H]3[C@@H]54)C1"]) ## tooks 11m 4s (!)

In [9]:
# group = admet_group(path = 'data/')
# benchmark = group.get("CYP2D6_Substrate_CarbonMangels")

# dataset = benchmark["test"]

# print(dataset[dataset.Drug == "CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C"].to_markdown())

# Output:
# |     | smiles                             |   Y |
# |----:|:-----------------------------------|----:|
# |  58 | CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C |   1 |
# | 226 | CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C |   0 |else 'regression'

In [5]:
import pathlib

calculated = set(map(str, pathlib.Path("embeddings/").glob("*/*.zip")))

pairs = product(get_dataset_names(), get_embedder_names())

with open("embedding_params.txt", "w") as f:
    for dataset_name, model_name in pairs:
        if f"embeddings/{dataset_name}/{model_name}.zip" not in calculated:
            print(dataset_name, model_name, file=f)


In [29]:
import pathlib
done = set(map(str, pathlib.Path("results/").glob("*.pkl")))

pairs = product(get_dataset_names(), get_embedder_names())

with open("predict_params.txt", "w") as f:
    for dataset_name, model_name in pairs:
        if (f"results/{dataset_name}_{model_name}_linear.pkl" not in done) or (f"results/{dataset_name}_{model_name}_forest.pkl" not in done):
            print(dataset_name, model_name, file=f)